In [1]:
#default_exp hypster_prepare

In [2]:
#export
from hypster.oo_hp import *

In [3]:
#export
from inspect import signature
import functools
from collections import OrderedDict

In [4]:
#export
def set_name(arg, name):
    if arg.manual_name == False:
        arg.name = name

In [5]:
#export
def set_name_from_arg(arg, name):
    if isinstance(arg, HpAtomic):
        set_name(arg, name)
    
    if isinstance(arg, HpToggle):
        set_name_from_arg(arg.hp, name)
        if arg.manual_name == False:
            arg.name = f"toggle_{get_name(arg.hp)}"
    
    if isinstance(arg, (list, tuple)):
        [set_name_from_arg(item, name) for item in arg]
    
    if isinstance(arg, HpVarLenIterable):
        set_name(arg, f"n_{name}")
        set_name_from_arg(arg.hp, name)
    
    if isinstance(arg, HpIterable):
        #TODO: continue
        list(set([item for item in arg]))
    
    if isinstance(arg, HypsterPrepare):
        if hasattr(arg.call, "__class__") and hasattr(arg.call.__class__, "__name__"):
            arg.name = f"{arg.call.__class__.__name__}_{arg.name}"

In [6]:
#TODO: handle HpExpressions
#TODO: handle HpIterable

In [7]:
a = HpInt(1, 6)
b = (a, 5, a-1)

In [8]:
#export
def set_names_from_args(arg_names, args, kwargs):
    # if argument is an iterable -> (start_mom, ..., ...) use its object name + counting index
    # fit_method = learner.fit_one_cycle(2, lr) --> during sampling, get the function signature and change the name
    for i, arg in enumerate(args):
        set_name_from_arg(arg, arg_names[i])
                
    for key, value in kwargs.items():
        set_name_from_arg(value, key)

In [9]:
#export
class HypsterPrepare(HypsterBase):
    def __init__(self, call, base_call, *args, **kwargs):
        #allow future option to add a prefix of a model name
        self.call            = call
        self.base_call       = base_call
        self.args            = args
        self.kwargs          = kwargs
        self.trials_sampled  = set()
        self.studies_sampled = set()
        self.base_object     = None
        self.result          = None
        
        if callable(call):
            self.arg_names = list(signature(call).parameters.keys())
            set_names_from_args(self.arg_names, self.args, self.kwargs)
        
    def sample(self, trial):
        if trial.study.study_name not in self.studies_sampled:
            self.trials_sampled = set()
        elif trial.number in self.trials_sampled:
            return self.result           
        
        if self.base_call is not None:
            self.base_object = self.base_call.sample(trial)

        self.sampled_args   = populate_iterable(self.args, trial)
        self.sampled_kwargs = populate_dict(self.kwargs, trial)
        
        self.trials_sampled.add(trial.number)
        self.studies_sampled.add(trial.study.study_name)

        if self.base_object:
            if len(self.sampled_args) == 0 and len(self.sampled_kwargs) == 0:
                self.result = getattr(self.base_object, self.call)
            else:
                self.result = getattr(self.base_object, self.call)(*self.sampled_args, **self.sampled_kwargs)
        else:
            self.result = self.call(*self.sampled_args, **self.sampled_kwargs)
        return self.result
    
    def __call__(self, *args, **kwargs):
        #print(f"args {args}, kwargs {kwargs}")
        self.args = args
        self.kwargs = kwargs
        return self
    
    def __getattr__(self, name, *args, **kwargs):
        #print(f"name {name}, args {args}, kwargs {kwargs}")
        return HypsterPrepare(name, self, *args, **kwargs)

In [10]:
#export
def prepare(call):
    @functools.wraps(call)
    def wrapper_decorator(*args, **kwargs):
        all_args = list(args) + list(kwargs.values())
        if any([contains_hypster(arg, HYPSTER_TYPES) for arg in all_args]):
            return HypsterPrepare(call, None, *args, **kwargs)
        else:
            return call(*args, **kwargs)
    return wrapper_decorator

# Test Naming

In [11]:
fill_strategy = HpCategorical(["mode", "median"])

In [12]:
class FillMissing:
    @auto_assign
    def __init__(self, fill_strategy="mode", add_col=False): pass

In [13]:
FillMissing = prepare(FillMissing)

In [14]:
imp = FillMissing(fill_strategy=fill_strategy, add_col=False)

In [15]:
signature(FillMissing).parameters

mappingproxy({'fill_strategy': <Parameter "fill_strategy='mode'">,
              'add_col': <Parameter "add_col=False">})

In [16]:
imp.kwargs["fill_strategy"].name

'fill_strategy'

# Test Prepare

In [17]:
def foo(a, b, c="he", d=None, **kwargs):
    if c is not None:
        print(f"{a} and {b} and {c}")
    else:
        print(f"{a} and {b}")
        
    return f"returned {a} and {b} and {c}"    

In [18]:
foo("hi", "helol", c="errr")

hi and helol and errr


'returned hi and helol and errr'

In [19]:
foo = prepare(foo)

In [20]:
a = foo("helo", HpCategorical(["go", "no"]))

In [21]:
foo("hi", HpCategorical(["hello", "hola!"]))

In [22]:
def foo2(a, b="hi!", c=None, **kwargs):
    if c is not None:
        print(f"{a} and {b} and {c}")
    else:
        print(f"{a} and {b}")
    
    return f"returned {a} and {b} and {c}"

In [23]:
class Cls:
    def __init__(self, a, b="hi!", c=None, **kwargs):
        if c is not None:
            print(f"{a} and {b} and {c}")
        else:
            print(f"{a} and {b}")

        #return f"returned {a} and {b} and {c}"
    
    def shmul(self, batch):
        return batch

In [24]:
Cls = prepare(Cls)

In [25]:
foo = prepare(foo)

In [26]:
hps = foo("hi", b="shmuli")

hi and shmuli and he


In [27]:
hps = foo("hi", b=HpBool())

In [28]:
hps = Cls("hi", b=HpCategorical(["Shmuli", "Buli"]))

In [29]:
hps.arg_names

['a', 'b', 'c', 'kwargs']

In [30]:
z = hps.shmul(batch=HpInt(1, 10))

In [31]:
#export
import optuna

In [32]:
#export
def run_func_test(x, n_trials=5):
    def objective(trial):
        print(x.sample(trial))
        return 1.0

    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=n_trials, timeout=600)

In [33]:
run_func_test(z)

hi and Shmuli
5
hi and Shmuli
6
hi and Shmuli
3
hi and Shmuli
10
hi and Shmuli
5


In [34]:
class Cls():
    def __init__(self, name, last_name="", nickname=""):
        self.name = name
        self.last_name = last_name
        self.nickname = nickname
        print(f"{self.name} {self.last_name} {self.nickname}")  

In [35]:
c = Cls("Gilad", nickname="The King!")

Gilad  The King!


In [36]:
Cls2 = prepare(Cls)

In [37]:
x = Cls2("Gilad", HpCategorical(["The King!", "The Best King!"]))

In [38]:
run_func_test(x)

Gilad The King! 
Gilad The King! 
Gilad The King! 
Gilad The Best King! 
Gilad The Best King! 


## HpToggle

In [39]:
Cls2 = prepare(Cls)

In [40]:
x = Cls2("Gilad", last_name=HpToggle(HpCategorical(["The King!", "The Best King!"])))

<class 'hypster.oo_hp.HpCategorical'>
<class 'hypster.oo_hp.HpCategorical'>


In [41]:
run_func_test(x)

Gilad The Best King! 
Gilad The King! 
Gilad The Best King! 
Gilad  
Gilad The Best King! 


In [42]:
from nbdev.export import notebook2script

In [43]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
